In [1]:
import pandas as pd

df = pd.read_csv("deanonymous_score_default.csv")

df["confidence"] = 0.5
df["trustiness"] = 0.5
df["reliability"] = 0.7

max_iterations = 1000
epsilon = 0.01

for iteration in range(max_iterations):
    df["trustiness_prev"] = df["trustiness"]
    df["reliability_prev"] = df["reliability"]
    df["confidence_prev"] = df["confidence"]
    df["weighted_score"] = df["deanonymous"] * df["confidence"]
    
    trustiness_updates = df.groupby("to")["weighted_score"].mean()
    df["trustiness"] = df["to"].map(trustiness_updates)

    reliability_updates = df.groupby("from")["confidence"].mean()
    df["reliability"] = df["from"].map(reliability_updates)

    df["trustiness"] = df["trustiness"].clip(0, 1)
    df["reliability"] = df["reliability"].clip(0, 1)

    df["confidence"] = (df["reliability"] + (1 - abs(df["deanonymous"] - df["trustiness"]))) / 2

    df["confidence"] = df["confidence"].clip(0, 1)

    delta_t = (df["trustiness"] - df["trustiness_prev"]).abs().sum()
    delta_r = (df["reliability"] - df["reliability_prev"]).abs().sum()
    delta_c = (df["confidence"] - df["confidence_prev"]).abs().sum()
    delta = max(delta_t, delta_r, delta_c)

    print(f"Iteration {iteration + 1}: ΔT = {delta_t:.4f}, ΔR = {delta_r:.4f}, ΔC = {delta_c:.4f}, Δ = {delta:.4f}")

    if delta < epsilon:
        print("Convergence achieved!")
        break

df["risk"] = (1 - df["reliability"]) * 10

Iteration 1: ΔT = 1674614.2421, ΔR = 827292.8000, ΔC = 647367.8617, Δ = 1674614.2421
Iteration 2: ΔT = 86656.5932, ΔR = 644070.0015, ΔC = 360411.2346, Δ = 644070.0015
Iteration 3: ΔT = 65718.0131, ΔR = 360399.8588, ΔC = 208898.6225, Δ = 360399.8588
Iteration 4: ΔT = 47292.4088, ΔR = 208878.8449, ΔC = 125089.2524, Δ = 208878.8449
Iteration 5: ΔT = 34099.7410, ΔR = 125053.3207, ΔC = 77868.3246, Δ = 125053.3207
Iteration 6: ΔT = 24768.3444, ΔR = 77801.1506, ΔC = 50459.3849, Δ = 77801.1506
Iteration 7: ΔT = 18163.4801, ΔR = 50409.6625, ΔC = 33836.5502, Δ = 50409.6625
Iteration 8: ΔT = 13424.2207, ΔR = 33803.1420, ΔC = 23300.2365, Δ = 33803.1420
Iteration 9: ΔT = 9975.8673, ΔR = 23258.2543, ΔC = 16405.9011, Δ = 23258.2543
Iteration 10: ΔT = 7459.0032, ΔR = 16356.6752, ΔC = 11743.3035, Δ = 16356.6752
Iteration 11: ΔT = 5587.4370, ΔR = 11707.0806, ΔC = 8528.1980, Δ = 11707.0806
Iteration 12: ΔT = 4203.7794, ΔR = 8501.1242, ΔC = 6287.2726, Δ = 8501.1242
Iteration 13: ΔT = 3183.8417, ΔR = 6265.

In [ ]:
risk_df = df.groupby("from")["risk"].mean().reset_index()

addresses = pd.read_csv("labeled_addresses.csv")
addresses = addresses[addresses["label"] == 1]["Address"]

merged_df = pd.merge(
    addresses.to_frame(name="address"),
    risk_df.rename(columns={"from": "address"}),
    on="address",
    how="left"
)

In [4]:
len(merged_df[merged_df['risk'] < 6])

37

In [5]:
len(merged_df[merged_df['risk'] >= 6])

198

In [6]:
len(merged_df[merged_df['risk'].isna()])

8

In [ ]:
risk_df = df.groupby("from")["risk"].mean().reset_index()

addresses = pd.read_csv("labeled_addresses.csv")
addresses = addresses[addresses["label"] == 0]["Address"]

merged_df = pd.merge(
    addresses.to_frame(name="address"),
    risk_df.rename(columns={"from": "address"}),
    on="address",
    how="left"
)

In [8]:
len(merged_df[merged_df['risk'] < 6])

457

In [9]:
len(merged_df[merged_df['risk'] >= 6])

79

In [10]:
len(merged_df[merged_df['risk'].isna()])

24